# RAG Hausaufgabe



## 0. Setup & Imports


In [ ]:
# TODO (easy): skim the imports and make sure you understand what each library is used for.

from dotenv import load_dotenv
import os
import glob
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss
from sentence_transformers import SentenceTransformer
import pickle
import random
import pandas as pd

# LLM / API clients (we will mainly use OpenAI here; Gemini can be added as a bonus)
from openai import OpenAI

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load API keys from .env (you need to create this file once and add your keys)
load_dotenv()

deepinfra_key = os.getenv("DEEPINFRA_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# For this exercise we mainly use OpenAI for both embeddings (RAG B) and chat completions.
assert openai_api_key is not None, "Please set OPENAI_API_KEY in your .env file."
openai_client = OpenAI(api_key= deepinfra_key, base_url="https://api.deepinfra.com/v1/openai")


In [3]:
# Make pandas show the full table and full cell content
pd.set_option("display.max_rows", None)       # show all rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.max_colwidth", None)   # don't truncate cell text

## 1. Load PDF documents




In [4]:
def load_pdfs(glob_path: str = "data/*.pdf") -> str:
    """Load all PDFs matching the pattern and return their combined text.

    TODO:
    - Use `glob.glob(glob_path)` to iterate over file paths
    - For each file, open it in binary mode and create a `PdfReader`
    - Loop over `reader.pages` and extract text with the extract_text() function
    - Concatenate everything into a single string `text`
    - Be robust: skip pages where `extract_text()` returns None
    """
    # YOUR CODE HERE
    text = ""
    for pdf_path in glob.glob(glob_path):
        print(pdf_path)
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += " " + page_text
    return text



In [5]:
# Run once and inspect
raw_text = load_pdfs("data/*.pdf")
print("Number of characters:", len(raw_text))
print("Preview:", raw_text[:500])

data/Cipralex.pdf
data/Candesartan.pdf
data/Aspirin.pdf
Number of characters: 139319
Preview:  Cipralex®
Lundbeck (Schweiz) AG
Zusammensetzung
Wirkstoffe
Filmtabletten, Tropfen zum Einnehmen, Lösung:  Escitalopramum ut escitaloprami oxalas
Hilfsstoffe
Filmtabletten:  Cellulosum microcristallinum silicificatum, Talcum, Carmellosum natricum conexum enthält ungefähr 0,32 mg (10 mg) oder 0,63 mg (20 mg)
natrium, Magnesii stearas, Hypromellosum, Macrogolum 400, E171
Tropfen zum Einnehmen, Lösung (20 mg/ml):  Acidum Citricum, Ethanolum 96 per centum 100 mg pro ml, Natrii hydroxidum corresp. ma


## 2. Chunk the text




In [6]:
# Base configuration (RAG A)
chunk_size = 2000
chunk_overlap = 200

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunks = splitter.split_text(raw_text)
print(f"RAG: {len(chunks)} chunks produced, first chunk length = {len(chunks[0])}")

RAG: 79 chunks produced, first chunk length = 1908


## 3. Create embeddings and a FAISS index




In [ ]:
# Embedding model (local)
model_name = "intfloat/e5-base-v2"
embedder = SentenceTransformer(model_name)


chunks_with_prefix = ["passage: " + chunk for chunk in chunks] #higher quality for this model
# Compute embeddings for all chunks of configuration A
embeddings = embedder.encode(chunks_with_prefix, convert_to_numpy=True)

dimensions = embeddings.shape[1]
print("Embedding dimensionality :", dimensions)

index = faiss.IndexFlatL2(dimensions)
index.add(embeddings)
print("FAISS index size:", index.ntotal)

# Persist index/chunks if you like (optional)
os.makedirs("faiss", exist_ok=True)
faiss.write_index(index, "faiss/faiss_index.index")
with open("faiss/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

Embedding dimensionality : 768
FAISS index size: 79


## 4. Implement a simple retriever




In [8]:
def retrieve_texts(query: str, k: int, index, chunks, embedder) -> list:
    """Return the top-k most similar chunks for a query.

    TODO (students):
    - Encode the query with `embedder.encode(...)`
    - Call `index.search(query_embedding, k)`
    - Use the returned indices to select the chunks
    - Return a list of strings (chunks)
    """
    # YOUR CODE HERE
    query_with_prefix = "query: " + query #higher quality for this model
    query_emb = embedder.encode([query_with_prefix], convert_to_numpy=True)
    distances, indices = index.search(query_emb, k)
    retrieved = [chunks[i] for i in indices[0]]
    return retrieved

# Quick sanity check
test_query = "Wie soll Cipralex gelagert werden?"
retrieved_text = retrieve_texts(test_query, k=3, index=index, chunks=chunks, embedder=embedder)
print("Number of retrieved chunks:", len(retrieved_text))
print("Preview of first chunk:", retrieved_text[0][:400])

Number of retrieved chunks: 3
Preview of first chunk: Cipralex®
Lundbeck (Schweiz) AG
Zusammensetzung
Wirkstoffe
Filmtabletten, Tropfen zum Einnehmen, Lösung:  Escitalopramum ut escitaloprami oxalas
Hilfsstoffe
Filmtabletten:  Cellulosum microcristallinum silicificatum, Talcum, Carmellosum natricum conexum enthält ungefähr 0,32 mg (10 mg) oder 0,63 mg (20 mg)
natrium, Magnesii stearas, Hypromellosum, Macrogolum 400, E171
Tropfen zum Einnehmen, Lösung


## 5. Implement `answer_query` (RAG + LLM)




In [9]:
def answer_query(query: str, k: int, index, chunks, embedder, client: OpenAI) -> str:
    """RAG-style answer: retrieve context and ask an LLM.

    TODO (students):
    - Use `retrieve_texts` to get `k` relevant chunks.
    - Join them into a single context string.
    - Build a chat prompt that instructs the model to answer *only* using the context.
    - Call `client.chat.completions.create(...)` with model `"Llama-3.3-70B-Instruct-Turbo"` (or similar).
    - Return the model's answer text.
    """
    retrieved_chunks = retrieve_texts(query, k, index, chunks, embedder)
    context = "\n\n---\n\n".join(retrieved_chunks)

    system_prompt = (
        """Du bist ein hilfreicher Assistent, der Fragen NUR basierend auf dem bereitgestellten Kontext beantwortet.
  Wenn die Antwort nicht im Kontext enthalten ist, sage dass du es nicht weisst.
  Antworte auf Deutsch."""
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Kontext:\n{context}\n\nFrage: {query}"}
    ]

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=messages
    )

    return completion.choices[0].message.content.strip()

# Quick manual test
answer = answer_query(test_query, k=3, index=index, chunks=chunks, embedder=embedder, client=openai_client)
print("RAG answer:", answer)

RAG answer: Cipralex sollte in der Originalverpackung und nicht über 30°C gelagert werden. Es sollte außerdem außer Reichweite von Kindern aufbewahrt werden.


## 6.  Testfragen für die Antwortfunktion Fragen

In [10]:
# Testfrage 1
frage1 = "Wie wird Candesartan normalerweise bei Erwachsenen dosiert?"
antwort1 = answer_query(frage1, k=3, index=index, chunks=chunks, embedder=embedder, client=openai_client)
print("Frage 1:", frage1)
print("Antwort:", antwort1)
print("\n" + "="*50 + "\n")

# Testfrage 2
frage2 = "Wer sollte Aspirin nicht anwenden? Nenne mindestens eine wichtige Gegenanzeige."
antwort2 =  answer_query(frage2, k=3, index=index, chunks=chunks, embedder=embedder, client=openai_client)
print("Frage 2:", frage2)
print("Antwort:", antwort2)

Frage 1: Wie wird Candesartan normalerweise bei Erwachsenen dosiert?
Antwort: Das liegt nicht im Kontext. Der Kontext beschreibt die Pharmakokinetik und -dynamik von Candesartan, sowie seine Wirkung bei verschiedenen Patientengruppen, aber es enthält keine Informationen über die Normaldosierung bei Erwachsenen.


Frage 2: Wer sollte Aspirin nicht anwenden? Nenne mindestens eine wichtige Gegenanzeige.
Antwort: Aspirin sollte nicht von Patienten mit Asthma bronchiale oder allgemeiner Neigung zu Überempfindlichkeit angewendet werden, da Acetylsalicylsäure Bronchospasmen begünstigen und Asthmaanfälle oder andere Überempfindlichkeitsreaktionen auslösen kann.


## 7. Fünf Fragen aus dem Kontext erstellen

In [13]:
def generate_questions_for_random_chunks(chunks, num_chunks: int = 5, max_retries: int = 2):
    selected_chunks = random.sample(chunks, num_chunks)
    qa_pairs = []

    for chunk in selected_chunks:
        prompt =  (
            "Formuliere eine aussagekräftige Frage, die die Kerninhalte des folgenden Textes abdeckt. Wichtig: Gib nur die Frage aus. Keine Einleitung, kein Zusatztext wie „Hier ist …“, keine Erklärung.\n\n"
            "Text:\n" + chunk + "\n\n"
            "Frage:"
        )

        question = None
        for attempt in range(max_retries):
            try:
                completion = openai_client.chat.completions.create(
                    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                    messages=[{"role": "user", "content": prompt}]
                )
                question = completion.choices[0].message.content.strip()
                if question:
                    break
            except Exception as e:
                print("Error while generating question, retrying...", e)

        if question is None:
            question = "Error: could not generate question."

        qa_pairs.append((chunk, question))

    return qa_pairs

questions = generate_questions_for_random_chunks(chunks=chunks, num_chunks=5, max_retries=2)
for i, (chunk, q) in enumerate(questions, 1):
    print(f"Q{i}: {q}\n  From chunk preview: {chunk[:120]}...\n")

Q1: Wie sollten Patienten mit Nieren- oder Leberfunktionsstörungen, ältere Patienten oder Kinder und Jugendliche die Kombination aus Candesartan und Amlodipin einnehmen?
  From chunk preview: Patienten, welche Candesartan und Amlodipin separat erhalten, können auf die entsprechende Dosis von Candesartan-Amlodip...

Q2: Welche Nebenwirkungen und Risiken können durch die Einnahme von Acetylsalicylsäure, insbesondere während der Schwangerschaft, Stillzeit und bei bestimmten Gesundheitszuständen, entstehen?
  From chunk preview: kardiopulmonale Toxizität (Verengung/vorzeitiger Verschluss des Ductus arteriosus und pulmonale Hypertonie);
Nierenfunkt...

Q3: Wie wirken sich die Wirkstoffe Candesartan und Amlodipin auf Schwangerschaft und Stillzeit aus und welche Empfehlungen gibt es für die Anwendung von Candesartan-Amlodipin-Mepha in diesen Lebensphasen?
  From chunk preview: Seite 4/11 Schwangerschaft/Stillzeit
Schwangerschaft
In Zusammenhang mit Candesartan-Amlodipin-Mepha:
Candesartan-Aml

## 7. Fünf Fragen beantworten

In [14]:
def answer_generated_questions(question_tuples, k, index, chunks, embedder, client):
    results = []
    for chunk, question in question_tuples:
        answer = answer_query(question, k, index, chunks, embedder, client)
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
    return results

results = answer_generated_questions(
    questions,
    k=5,
    index=index,
    chunks=chunks,
    embedder=embedder,
    client=openai_client,
)

for item in results:
    print("Question:", item["question"])
    print("Answer :", item["answer"])
    print("Source chunk preview:", item["chunk"][:150], "...")
    print("-" * 60)

Question: Wie sollten Patienten mit Nieren- oder Leberfunktionsstörungen, ältere Patienten oder Kinder und Jugendliche die Kombination aus Candesartan und Amlodipin einnehmen?
Answer : Patienten mit Nieren- oder Leberfunktionsstörungen, ältere Patienten sowie Kinder und Jugendliche sollten die Kombination aus Candesartan und Amlodipin wie folgt einnehmen:

- Patienten mit leichter bis mittelschwerer Leberinsuffizienz sollten Candesartan-Amlodipin-Mepha mit Vorsicht einnehmen. Bei schwerer Leberinsuffizienz und/oder Cholestase ist es kontraindiziert.
- Patienten mit leichter bis mittelschwerer Niereninsuffizienz benötigen keine Dosisanpassung, aber bei mittelschwerer Niereninsuffizienz wird eine Überwachung des Kaliumspiegels und des Kreatinins empfohlen. Bei schwerer Niereninsuffizienz oder Niereninsuffizienz im Endstadium ist Vorsicht geboten.
- Ältere Patienten benötigen keine Dosisanpassung, aber bei Dosiserhöhungen ist Vorsicht geboten.
- Für Kinder und Jugendliche unter 18 Jahren 